# Network


In [3]:
import matplotlib.pyplot as plt
import tensorflow as tf
import tqdm
import numpy as np
#tf.reset_default_graph()

# these should go into ops.py

In [235]:
!pwd

/Users/ostyk/Desktop/self-driving-AttGAN/code


In [228]:
def conv2d(input_, output_dim, k_h=5, k_w=5, d_h=2, d_w=2, stddev=0.02, name="conv2d", padding = 'VALID'):
    with tf.variable_scope(name):
        w = tf.get_variable('w', [k_h, k_w, input_.get_shape()[-1], output_dim],
              initializer=tf.truncated_normal_initializer(stddev=stddev))
        
        conv = tf.nn.conv2d(input_, w, strides=[1, d_h, d_w, 1], padding=padding)
        biases = tf.get_variable('biases', [output_dim], initializer=tf.constant_initializer(0.0))

        return tf.reshape(tf.nn.bias_add(conv, biases), conv.get_shape())

    
def deconv2d(input_, output_shape, k_h=5, k_w=5, d_h=2, d_w=2, name="deconv2d", stddev=0.02, with_w=False):
    with tf.variable_scope(name):
        # filter : [height, width, output_channels, in_channels]
        w = tf.get_variable('w', [k_h, k_w, output_shape[-1], input_.get_shape()[-1]],
                            initializer=tf.random_normal_initializer(stddev=stddev))


        deconv = tf.nn.conv2d_transpose(input_, w, output_shape=output_shape, strides=[1, d_h, d_w, 1])

        biases = tf.get_variable('biases', [output_shape[-1]], initializer=tf.constant_initializer(0.0))
        
        return tf.reshape(tf.nn.bias_add(deconv, biases), deconv.get_shape())


    
def encoder(inputs, name = 'G_encoder', reuse=tf.AUTO_REUSE, is_training = True):
    """
    encoder function
    :param: inputs
    :param: name
    :return list of layers:
    """
    leakyrelu_alpha = 0.2
    num_blocks = 5
    filters = 64
    kernel_size = 4
    strides = 2
    layers = [inputs]
    
    with tf.variable_scope(name, reuse=reuse):
        
        for i in range(num_blocks):
            
            conv = conv2d(inputs, filters, kernel_size, kernel_size, strides, strides, name = str(i+1))
            batch_norm = tf.contrib.layers.batch_norm(conv)
            inputs = leaky_relu = tf.nn.leaky_relu(batch_norm, alpha = leakyrelu_alpha)
            
            filters += filters
            layers.append(inputs)
            
        return layers
    
def decoder(inputs, label, name = 'G_decoder', reuse=None, is_training = True):
    """
    decoder function
    :param: inputs (list of layers from encoder)
    :param: name
    :return tanh(conv5):
    """
    leakyrelu_alpha = 0.2
    filters = 1024
    kernel_size = 4
    strides = 2
    input_ = inputs[-1]

    
    with tf.variable_scope(name, reuse=reuse):
                
        for ind in list(reversed(range(len(inputs)))):
            outout_shape = [i.value for i in list(inputs[ind-1].shape)]
            
            if ind==1:
                deconv = deconv2d(input_, outout_shape, kernel_size, kernel_size, strides, strides, name = "deconv_{}".format(ind))
                return tf.nn.tanh(deconv)
            
            deconv = deconv2d(input_, outout_shape, kernel_size, kernel_size, strides, strides, name = str(ind-1))
            concatenated = tf.concat([deconv, inputs[ind-1]], axis=3)

            batch_norm = tf.contrib.layers.batch_norm(concatenated)
            
            input_ = leaky_relu = tf.nn.leaky_relu(batch_norm, alpha = leakyrelu_alpha, name = "ReLU_{}".format(ind))

In [229]:
IMG_WIDTH

128

In [230]:
IMG_WIDTH = 128
IMG_HEIGHT = 128
BATCH_SIZE = 32
NUM_CLASSES = 2
tf.reset_default_graph()
tf.compat.v1.disable_eager_execution()

input_enc = tf.placeholder(tf.float32,shape=[BATCH_SIZE,IMG_WIDTH,IMG_HEIGHT,3],name="input_scene")
input_dec = tf.placeholder(tf.float32,shape=[BATCH_SIZE,IMG_WIDTH,IMG_HEIGHT,3],name="z")
label = tf.placeholder(tf.float32,shape=[BATCH_SIZE, NUM_CLASSES],name="input_label")

gen_enc = encoder(input_enc, reuse=tf.AUTO_REUSE)
gen_enc

[<tf.Tensor 'input_scene:0' shape=(32, 128, 128, 3) dtype=float32>,
 <tf.Tensor 'G_encoder/LeakyRelu:0' shape=(32, 63, 63, 64) dtype=float32>,
 <tf.Tensor 'G_encoder/LeakyRelu_1:0' shape=(32, 30, 30, 128) dtype=float32>,
 <tf.Tensor 'G_encoder/LeakyRelu_2:0' shape=(32, 14, 14, 256) dtype=float32>,
 <tf.Tensor 'G_encoder/LeakyRelu_3:0' shape=(32, 6, 6, 512) dtype=float32>,
 <tf.Tensor 'G_encoder/LeakyRelu_4:0' shape=(32, 2, 2, 1024) dtype=float32>]

In [231]:
# Generator decoder input and instance
Z_SHAPE = gen_enc[-1].shape
N_layers = len(gen_enc)
input_dec = tf.placeholder(tf.float32,shape=[N_layers, BATCH_SIZE,Z_SHAPE[1].value,Z_SHAPE[2].value,Z_SHAPE[3].value],name="z")
label = tf.placeholder(tf.float32,shape=[BATCH_SIZE, NUM_CLASSES],name="input_label")



In [232]:
gen_dec = decoder(gen_enc, label, reuse=tf.AUTO_REUSE)

In [233]:
gen_dec

<tf.Tensor 'G_decoder/Tanh:0' shape=(32, 128, 128, 3) dtype=float32>

In [168]:
def discriminator(input_var, name, reuse=None):
    with tf.variable_scope(name,reuse=reuse):
        leakyrelu_alpha = 0.2
        num_blocks = 5
        filters = 64
        kernel_size = 4
        strides = 2
        # Five intermediate blocks : conv + layer norm + instance norm + leaky relu
        for i in range(num_blocks):
            conv = tf.layers.conv2d(inputs = input_var,
                                    filters = filters, 
                                    kernel_size = kernel_size,
                                    padding = 'valid', 
                                    strides = strides,
                                    name = "{}_{}_{}".format(name, conv.__name__, i+1))
            layer_norm = tf.contrib.layers.layer_norm(conv, name = "{}_{}_{}".format(name, layer_norm.__name__, i+1))
            instance_norm = tf.contrib.layers.instance_norm(layer_norm, name = "{}_{}_{}".format(name, instance_norm.__name__, i+1))
            leaky_relu_out = tf.nn.leaky_relu(instance_norm, alpha = leakyrelu_alpha)

            input_var = leaky_relu_out
            filters += filters

        # Output block : fc(1024) + layer norm + instance norm + leaky relu
        output_blocks = input_var
        fc = tf.contrib.layers.fully_connected(output_blocks, 
                                               num_outputs = 1024,
                                               name = "{}_{}_{}".format(name, fc.__name__, "out"))
        layer_norm = tf.contrib.layers.layer_norm(fc,
                                                  name = "{}_{}_{}".format(layer_norm, layer_.__name__, "out"))
        instance_norm = tf.contrib.layers.instance_norm(layer_norm, 
                                                        name = "{}_{}_{}".format(name, instance_norm.__name__, i+1))
        leaky_relu_out = tf.nn.leaky_relu(instance_norm, alpha = leakyrelu_alpha)

        # Output
        return tf.contrib.layers.fully_connected(leaky_relu_out, num_outputs = 1)

def classifier(input_var, name, reuse=None):
    with tf.variable_scope(name,reuse=reuse):
        leakyrelu_alpha = 0.2
        num_blocks = 5
        filters = 64
        kernel_size = 4
        strides = 2
        
        # Five intermediate blocks : conv + layer norm + instance norm + leaky relu
        for i in range(num_blocks):
            conv = tf.layers.conv2d(inputs = input_var, 
                                    filters = filters,
                                    kernel_size = kernel_size, 
                                    padding = 'valid', 
                                    strides = strides,
                                    name = "{}_{}_{}".format(name, conv.__name__, i+1))
            layer_norm = tf.contrib.layers.layer_norm(conv)
            instance_norm = tf.contrib.layers.instance_norm(layer_norm)
            leaky_relu_out = tf.nn.leaky_relu(instance_norm, alpha = leakyrelu_alpha)

            input_var = leaky_relu_out
            filters += filters

        # Output block : fc(1024) + layer norm + instance norm + leaky relu
        output_blocks = input_var
        fc = tf.contrib.layers.fully_connected(output_blocks, num_outputs=1024)
        layer_norm = tf.contrib.layers.layer_norm(fc)
        instance_norm = tf.contrib.layers.instance_norm(layer_norm)
        leaky_relu_out = tf.nn.leaky_relu(instance_norm, alpha = leakyrelu_alpha)

        # Output
        out = tf.contrib.layers.fully_connected(leaky_relu_out, num_outputs=2)
        
        return tf.nn.sigmoid(out)